CARGAR LOS DATOS

In [116]:
from h5py import File
from numpy import array
import biosignalsnotebooks as bsnb
import matplotlib.pyplot as plt
from bokeh.plotting import figure, show
from numpy import cumsum, concatenate, zeros, linspace, average, power, absolute, mean, std, max, array, diff, where
from scipy.integrate import cumtrapz
from scipy.signal import welch
import math
import numpy as np

In [117]:
File_Name="signalbicep.h5"

h5_object=File(File_Name)

In [118]:
data, header=bsnb.load("signalbicep.h5",get_header=True)

MOSTRAR LOS DATOS

In [119]:
signal=data["CH1"]
time=bsnb.generate_time(signal,header["sampling rate"])

In [120]:
bokeh_figure=figure(x_axis_label="Time(s)", y_axis_label="Raw Data")

In [121]:
bokeh_figure.line(time,signal,legend_label="Datos Originales")

GlyphRenderer(id='4450', ...)

In [122]:
show(bokeh_figure)

DETECCION DE EVENTOS

In [123]:
from scipy.signal import butter, lfilter
from scipy.stats import linregress

In [124]:
pre_pro_signal=signal-average(signal)
sr = header["sampling rate"]
#Filtrado
low_cutoff=10 
high_cutoff=300

In [125]:
bokeh_figure=figure(x_axis_label="Time(s)", y_axis_label="Raw Data")
bokeh_figure.line(time,pre_pro_signal,legend_label="Datos Filtrados")
show(bokeh_figure)

In [126]:
avg_pre_pro_signal=average(pre_pro_signal)
std_pre_pro_signal=std(pre_pro_signal)

In [127]:
def normReg(thresholdLevel):
    threshold_0_perc_level = (- avg_pre_pro_signal) / float(std_pre_pro_signal)
    threshold_100_perc_level = (max(smooth_signal) - avg_pre_pro_signal) / float(std_pre_pro_signal)
    m, b = linregress([0, 100], [threshold_0_perc_level, threshold_100_perc_level])[:2]
    return m * thresholdLevel + b

threshold_level=10
threshold_level_norm_10=normReg(threshold_level)

threshold_level2=80
threshold_level_norm_80=normReg(threshold_level2)

threshold_10=avg_pre_pro_signal+threshold_level_norm_10*std_pre_pro_signal
threshold_80=avg_pre_pro_signal+threshold_level_norm_80*std_pre_pro_signal

In [128]:
rect_signal = absolute(pre_pro_signal)

# Smoothing level [Size of sliding window]
smoothing_level_perc = 20 # Percentage.
smoothing_level = int((smoothing_level_perc / 100) * sr)

#Smooth the signal
smooth_signal = bsnb.smooth(rect_signal, smoothing_level, window='hanning')

bsnb.plot(time, smooth_signal)
binary_signal=[]
for i in range(0,len(time)):
    if smooth_signal[i]>=threshold_10:
        binary_signal.append(1)
    else:
        binary_signal.append(0)



EXTRACIÓN DE CARACTERISTICAS

In [129]:
device = header["device"]
channel = "CH1" 
resolution = header["resolution"][0]

#muestreo y frecuencias
fs=header["sampling rate"]
signal=bsnb.raw_to_phy("EMG",device,data[channel],resolution,option="mV")
time=linspace(0,len(signal)/fs,len(signal))

In [130]:
burst_begin,burst_end=bsnb.detect_emg_activations(signal,fs,smooth_level=20,threshold_level=10, time_units=True, plot_result=True)[:2]

In [134]:
diff_signal = np.diff(binary_signal)
act_begin = np.where(diff_signal == 1)[0]
act_end = np.where(diff_signal == -1)[0]
print(act_begin)
print(act_end)

[171]
[6668]


In [ ]:
#Maximo
max_sample_value=max(signal)
#minimo
min_sample_value=min(signal)

avg_sample_value=average(signal)
std_sample_value=std(signal)

time_param_dict={"Maximum EMG":max_sample_value,"Minimum EMG":min_sample_value, "Average EMG":avg_sample_value,"Standard Deviation EMG":std_sample_value}
print(time_param_dict)

{'Maximum EMG': 207.4764618434093, 'Minimum EMG': -1.6352824578790883, 'Average EMG': 101.42870674347539, 'Standard Deviation EMG': 32.44058343217785}


In [ ]:
#root mean square
rms=sqrt(sum(signal*signal)/len(signal))

#area bajo la curva
area=cumtrapz(signal)
print(rms)
print(area)

106.4902530989293
[1.01285307e+02 2.02468409e+02 3.03753717e+02 ... 7.29981504e+05
 7.30083096e+05 7.30184892e+05]


In [ ]:
f,P=welch(signal,fs=fs,window="hanning", noverlap=0,nfft=int(256,))

#Potencia total y frecuencia media
area_freq=cumtrapz(P,f,initial=0)
total_power=area_freq[-1]
median_freq=f[where(area_freq>=total_power/2)[0][0]]
f_max=f[argmax(P)]

print(area_freq)
print(total_power)
print(median_freq)
print(f_max)

[0.00000000e+00 1.13940586e-01 1.98540225e-01 3.72772739e-01
 9.18466731e-01 2.50844458e+00 6.22788356e+00 1.48471016e+01
 2.87051820e+01 5.05226417e+01 9.44534537e+01 1.61331597e+02
 2.28549436e+02 2.86904649e+02 3.38604038e+02 3.83616910e+02
 4.29345116e+02 4.74268531e+02 5.13729016e+02 5.56238558e+02
 6.06061608e+02 6.63312648e+02 7.22952425e+02 7.70450247e+02
 8.05930074e+02 8.34132021e+02 8.56064099e+02 8.74570567e+02
 8.88499468e+02 8.99466537e+02 9.11385013e+02 9.23593964e+02
 9.33221049e+02 9.40103516e+02 9.45688615e+02 9.50313657e+02
 9.53936960e+02 9.57044090e+02 9.59406633e+02 9.61247579e+02
 9.62884369e+02 9.64548187e+02 9.66817900e+02 9.69779413e+02
 9.72610888e+02 9.75160824e+02 9.77584848e+02 9.80006291e+02
 9.82123057e+02 9.83667942e+02 9.85374084e+02 9.87597443e+02
 9.89761742e+02 9.91587493e+02 9.93332794e+02 9.95155089e+02
 9.97295615e+02 9.99065446e+02 1.00015920e+03 1.00126574e+03
 1.00263401e+03 1.00385356e+03 1.00476763e+03 1.00566154e+03
 1.00651551e+03 1.007252